# Transformações e Padronizações – Camada Trusted

Este notebook realiza a transformação dos dados provenientes da camada RAW, aplicando regras de limpeza, normalização, padronização de tipos, renomeação de colunas e modelagem estrutural. O objetivo é gerar datasets confiáveis, prontos para análises, integração e futura carga na camada Refined ou Gold.

As tabelas tratadas incluem: **empresas**, **estabelecimentos**, **sócios**, **simples_nacional**, **CNAE**, **municípios**, **natureza_jurídica**, **motivos** e **países**.

In [72]:
# Para iniciar a seção spark
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DecimalType
from pyspark.sql import functions as f
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

In [73]:
builder = SparkSession.builder \
    .appName("App Preparação RAW") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g")

spark: SparkSession = configure_spark_with_delta_pip(builder).getOrCreate()

## TRUSTED - Municipios

### Etapa 1: Leitura da camada RAW

In [ ]:
# Carrega dados da camada RAW
deltaTable_municipios = DeltaTable.forPath(spark, "../RAW/municipios")
df_municipios_raw = deltaTable_municipios.toDF()
df_municipios_raw.show(truncate=False)

+-------+------------------------+----------------------------------------------------------------------------------------+-----------------------------------------------------------------------------+
|id     |nome                    |microrregiao                                                                            |regiao-imediata                                                              |
+-------+------------------------+----------------------------------------------------------------------------------------+-----------------------------------------------------------------------------+
|1100015|Alta Floresta D'Oeste   |{11006, Cacoal, {1102, Leste Rondoniense, {11, RO, Rondônia, {1, N, Norte}}}}           |{110005, Cacoal, {1102, Ji-Paraná, {11, RO, Rondônia, {1, N, Norte}}}}       |
|1100023|Ariquemes               |{11003, Ariquemes, {1102, Leste Rondoniense, {11, RO, Rondônia, {1, N, Norte}}}}        |{110002, Ariquemes, {1101, Porto Velho, {11, RO, Rondônia, {1, N, Nor

### Etapa 2: Transformação dos dados

- Flatten dos campos aninhados

- Renomeação das colunas

- Padronização de nomes para o padrão Trusted

In [ ]:
df_municipios_trusted = df_municipios_raw.select(
    f.col("id").alias("id_municipio"),
    f.col("nome").alias("nome_municipio"),

    f.col("microrregiao.id").alias("id_microrregiao"),
    f.col("microrregiao.nome").alias("nome_microrregiao"),

    f.col("microrregiao.mesorregiao.id").alias("id_mesorregiao"),
    f.col("microrregiao.mesorregiao.nome").alias("nome_mesorregiao"),

    f.col("microrregiao.mesorregiao.UF.id").alias("id_uf"),
    f.col("microrregiao.mesorregiao.UF.sigla").alias("sigla_uf"),
    f.col("microrregiao.mesorregiao.UF.nome").alias("nome_uf"),

    f.col("microrregiao.mesorregiao.UF.regiao.id").alias("id_regiao"),
    f.col("microrregiao.mesorregiao.UF.regiao.sigla").alias("sigla_regiao"),
    f.col("microrregiao.mesorregiao.UF.regiao.nome").alias("nome_regiao")
)

In [19]:
df_municipios_trusted.show(5, truncate=False)
df_municipios_trusted.printSchema()

+------------+---------------------+---------------+-----------------+--------------+-----------------+-----+--------+--------+---------+------------+-----------+
|id_municipio|nome_municipio       |id_microrregiao|nome_microrregiao|id_mesorregiao|nome_mesorregiao |id_uf|sigla_uf|nome_uf |id_regiao|sigla_regiao|nome_regiao|
+------------+---------------------+---------------+-----------------+--------------+-----------------+-----+--------+--------+---------+------------+-----------+
|1100015     |Alta Floresta D'Oeste|11006          |Cacoal           |1102          |Leste Rondoniense|11   |RO      |Rondônia|1        |N           |Norte      |
|1100023     |Ariquemes            |11003          |Ariquemes        |1102          |Leste Rondoniense|11   |RO      |Rondônia|1        |N           |Norte      |
|1100031     |Cabixi               |11008          |Colorado do Oeste|1102          |Leste Rondoniense|11   |RO      |Rondônia|1        |N           |Norte      |
|1100049     |Cacoal  

### Etapa 3: Escrita na camada Trusted (TRS)

- Particionado por: sigla_uf

In [ ]:
try:
    df_municipios_trusted.write.format("delta") \
        .mode("overwrite") \
        .partitionBy("sigla_uf") \
        .save("../TRS/municipios")
    print("Dados salvo!")
except Exception as e:
    print(e)

Dados salvo!


### Etapa 4: Ajuste e Validação

- Releitura para verificação

In [54]:
try:
    deltaTable_municipios = DeltaTable.forPath(spark, "../TRS/municipios")
    df_municipios_trusted = deltaTable_municipios.toDF()
    df_municipios_trusted.show(5, truncate=False)
    df_municipios_trusted.printSchema()
except Exception as e:
    print(f"Erro: {e}")

+------------+------------------+---------------+------------------------+--------------+----------------------+-----+--------+-------+---------+------------+-----------+
|id_municipio|nome_municipio    |id_microrregiao|nome_microrregiao       |id_mesorregiao|nome_mesorregiao      |id_uf|sigla_uf|nome_uf|id_regiao|sigla_regiao|nome_regiao|
+------------+------------------+---------------+------------------------+--------------+----------------------+-----+--------+-------+---------+------------+-----------+
|2200053     |Acauã             |22015          |Alto Médio Canindé      |2204          |Sudeste Piauiense     |22   |PI      |Piauí  |2        |NE          |Nordeste   |
|2200103     |Agricolândia      |22005          |Médio Parnaíba Piauiense|2202          |Centro-Norte Piauiense|22   |PI      |Piauí  |2        |NE          |Nordeste   |
|2200202     |Água Branca       |22005          |Médio Parnaíba Piauiense|2202          |Centro-Norte Piauiense|22   |PI      |Piauí  |2        |

- Validação de nulos (colunas)

In [53]:
try:
    df_municipios_trusted.select([
        f.sum(f.col(c).isNull().cast("int")).alias(c)
        for c in df_municipios_trusted.columns
    ]).show()
except Exception as e:
    print(f"Erro: {e}")

+------------+--------------+---------------+-----------------+--------------+----------------+-----+--------+-------+---------+------------+-----------+
|id_municipio|nome_municipio|id_microrregiao|nome_microrregiao|id_mesorregiao|nome_mesorregiao|id_uf|sigla_uf|nome_uf|id_regiao|sigla_regiao|nome_regiao|
+------------+--------------+---------------+-----------------+--------------+----------------+-----+--------+-------+---------+------------+-----------+
|           0|             0|              0|                0|             0|               0|    0|       0|      0|        0|           0|          0|
+------------+--------------+---------------+-----------------+--------------+----------------+-----+--------+-------+---------+------------+-----------+



- Correção de valores nulos pontuais

In [25]:
try:
    # Remoção de NULL's
    deltaTable_municipios.update(
        condition = f.col("id_municipio") == "5101837",
        set = { "sigla_uf": f.lit("MT") }
    )
    print("Coluna atualizada com sucesso!")
except Exception as e:
    print(f"Erro: {e}")

25/07/18 10:22:04 WARN UpdateCommand: Could not validate number of records due to missing statistics.


Coluna atualizada com sucesso!


## TRUSTED - Natureza Jurídica

### Etapa 1: Leitura da camada RAW

In [44]:
try:
    deltaTable_natureza_juridica_raw = DeltaTable.forPath(spark, "../RAW/natureza_juridica")
    df_natureza_juridica_raw = deltaTable_natureza_juridica_raw.toDF()
    df_natureza_juridica_raw.show(5, truncate=False)
    df_natureza_juridica_raw.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+------------------------+------------------------------------------+
|codigo_natureza_juridica|natureza_juridica                         |
+------------------------+------------------------------------------+
|0000                    |Natureza Jurídica não informada           |
|3271                    |Órgão de Direção Local de Partido Político|
|3280                    |Comitê Financeiro de Partido Político     |
|3298                    |Frente Plebiscitária ou Referendária      |
|3301                    |Organização Social (OS)                   |
+------------------------+------------------------------------------+
only showing top 5 rows
root
 |-- codigo_natureza_juridica: string (nullable = true)
 |-- natureza_juridica: string (nullable = true)



### Etapa 2: Escrita na camada Trusted (TRS)

In [45]:
try:
    # Salvando dados no TRS, não precisou de alterações
    df_natureza_juridica_raw.write.format("delta") \
        .mode("overwrite") \
        .save("../TRS/natureza_juridica")
except Exception as e:
    print(f"Erro: {e}")

### Etapa 3: Ajuste e Validação

In [51]:
try:
    deltaTable_natureza_juridica_trs = DeltaTable.forPath(spark, "../TRS/natureza_juridica")
    df_natureza_juridica_trs = deltaTable_natureza_juridica_trs.toDF()
    df_natureza_juridica_trs.orderBy(df_natureza_juridica_trs["natureza_juridica"].desc()).show(100, truncate=False)
    df_natureza_juridica_trs.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+------------------------+------------------------------------------------------------------------+
|codigo_natureza_juridica|natureza_juridica                                                       |
+------------------------+------------------------------------------------------------------------+
|3263                    |Órgão de Direção Regional de Partido Político                           |
|3255                    |Órgão de Direção Nacional de Partido Político                           |
|3271                    |Órgão de Direção Local de Partido Político                              |
|1066                    |Órgão Público do Poder Legislativo Municipal                            |
|1040                    |Órgão Público do Poder Legislativo Federal                              |
|1058                    |Órgão Público do Poder Legislativo Estadual ou do Distrito Federal      |
|1074                    |Órgão Público do Poder Judiciário Federal                               |


In [67]:
try:
    # Filtragem de dados em busca de nulls
    df_natureza_juridica_raw.select([
        f.sum(f.col(c).isNull().cast("int")).alias(c)
        for c in df_natureza_juridica_raw.columns
    ]).show()
except Exception as e:
    print(f"Erro: {e}")

+------------------------+-----------------+
|codigo_natureza_juridica|natureza_juridica|
+------------------------+-----------------+
|                       0|                0|
+------------------------+-----------------+



## TRUSTED - CNAE

### Etapa 1: Leitura da camada RAW

In [ ]:
try:
    deltaTable_cnae: DeltaTable = DeltaTable.forPath(spark, "../RAW/cnae")
    df_cnae_raw: DataFrame = deltaTable_cnae.toDF()
    df_cnae_raw.show(5, truncate=False)
except Exception as e:
    print(f"Erro: {e}")

+-----------+---------------------------------------------------------+
|codigo_cnae|descricao_cnae                                           |
+-----------+---------------------------------------------------------+
|0111301    |Cultivo de arroz                                         |
|0111302    |Cultivo de milho                                         |
|0111303    |Cultivo de trigo                                         |
|0111399    |Cultivo de outros cereais não especificados anteriormente|
|0112101    |Cultivo de algodão herbáceo                              |
+-----------+---------------------------------------------------------+
only showing top 5 rows


### Etapa 2: Escrita na camada Trusted (TRS)

In [72]:
try:
    df_cnae_raw.write.format("delta") \
        .mode("overwrite") \
        .save("../TRS/cnae")
except Exception as e:
    print(f"Erro: {e}")

### Etapa 3: Ajuste e Validação

In [85]:
try:
    deltaTable_cnae: DeltaTable = DeltaTable.forPath(spark, "../TRS/cnae")
    df_cnae_trusted: DataFrame = deltaTable_cnae.toDF()
    df_cnae_trusted.show(5, truncate=False)
    df_cnae_trusted.printSchema()
except Exception as e:
    print(f"Erro: {e}")

+-----------+---------------------------------------------------------+
|codigo_cnae|descricao_cnae                                           |
+-----------+---------------------------------------------------------+
|0111301    |Cultivo de arroz                                         |
|0111302    |Cultivo de milho                                         |
|0111303    |Cultivo de trigo                                         |
|0111399    |Cultivo de outros cereais não especificados anteriormente|
|0112101    |Cultivo de algodão herbáceo                              |
+-----------+---------------------------------------------------------+
only showing top 5 rows
root
 |-- codigo_cnae: string (nullable = true)
 |-- descricao_cnae: string (nullable = true)



In [75]:
try:
    df_cnae_trusted.select([
        f.sum(f.col(c).isNull().cast("int")).alias(c)
        for c in df_cnae_trusted.columns
    ]).show()
except Exception as e:
    print(f"Erro: {e}")

+-----------+--------------+
|codigo_cnae|descricao_cnae|
+-----------+--------------+
|          0|             0|
+-----------+--------------+



## TRUSTED - Qualificação dos Sócios

### Etapa 1: Leitura da Camada RAW

In [81]:
try:
    deltaTable_qualificacao = DeltaTable.forPath(spark, "../RAW/qualificacao_socios")
    df_qualificacao_socios_raw = deltaTable_qualificacao.toDF()
    df_qualificacao_socios_raw.show(5, truncate=False)
    df_qualificacao_socios_raw.printSchema()
except Exception as e:
    print(f"Erro ao carregar dados: {e}")

+-------------------+----------------------------+
|codigo_qualificacao|descricao_qualificacao      |
+-------------------+----------------------------+
|00                 |Não informada               |
|05                 |Administrador               |
|08                 |Conselheiro de Administração|
|09                 |Curador                     |
|10                 |Diretor                     |
+-------------------+----------------------------+
only showing top 5 rows
root
 |-- codigo_qualificacao: string (nullable = true)
 |-- descricao_qualificacao: string (nullable = true)



### Etapa 2: Escrita na Camada Trusted (TRS)

In [82]:
try:
    df_qualificacao_socios_raw.write.format("delta") \
        .mode("overwrite") \
        .save("../TRS/qualificacao_socios")
    print("Dados salvos com sucesso na camada Trusted!")
except Exception as e:
    print(f"Erro ao salvar dados: {e}")

Dados salvos com sucesso na camada Trusted!


### Etapa 3: Ajuste e Validação

In [23]:
try:
    deltaTable_trusted = DeltaTable.forPath(spark, "../TRS/qualificacao_socios")
    df_qualificacao_socios_trusted = deltaTable_trusted.toDF()

    df_qualificacao_socios_trusted.show()
    df_qualificacao_socios_trusted.printSchema()
except Exception as e:
    print(f"Erro na validação: {e}")

+-------------------+----------------------+
|codigo_qualificacao|descricao_qualificacao|
+-------------------+----------------------+
|                 00|         Não informada|
|                 05|         Administrador|
|                 08|  Conselheiro de Ad...|
|                 09|               Curador|
|                 10|               Diretor|
|                 11|           Interventor|
|                 12|         Inventariante|
|                 13|            Liquidante|
|                 14|                   Mãe|
|                 15|                   Pai|
|                 16|            Presidente|
|                 17|            Procurador|
|                 18|            Secretário|
|                 19|  Síndico (Condomínio)|
|                 20|  Sociedade Consorc...|
|                 21|     Sociedade Filiada|
|                 22|                 Sócio|
|                 23|     Sócio Capitalista|
|                 24|     Sócio Comanditado|
|         

In [ ]:
try:
    # Verificar valores nulos em todas as colunas
    df_qualificacao_socios_trusted.select([
        f.sum(f.col(c).isNull().cast("int")).alias(c)
        for c in df_qualificacao_socios_trusted.columns
    ]).show()
except Exception as e:
    print(f"Erro: {e}")

## TRUSTED - Empresas

###  Etapa 1: Leitura da Camada RAW

In [ ]:
try:
    deltaTable_empresas_raw = DeltaTable.forPath(spark, "../RAW/empresas")
    df_empresas_raw = deltaTable_empresas_raw.toDF()
    df_empresas_raw.show(5, truncate=False)
    df_empresas_raw.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+-----------+---------------------------------------------------------------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+
|cnpj_basico|razao_social                                                         |codigo_natureza_juridica|codigo_qualificacao_responsavel|capital_social|codigo_porte_empresa|ente_federativo_responsavel|
+-----------+---------------------------------------------------------------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+
|08127452   |ELEICAO 2006 WALDEMAR ALBERTO BORGES RODRIGUES NETO DEPUTADO ESTADUAL|4090                    |51                             |0,00          |05                  |NULL                       |
|08127453   |ELEICAO 2006 JOSE ROBERTO DOS SANTOS PALADINI DEPUTADO ESTADUAL      |4090                    |51                             |0,00          |05                  |NULL

### Etapa 2: Transformação dos Dados

In [ ]:
df_empresas_raw = df_empresas_raw.withColumn(
    'capital_social',
    f.regexp_replace('capital_social', ',', '.')
)

df_empresas_raw.show(5, truncate=False)
df_empresas_raw.printSchema()

+-----------+---------------------------------------------------------------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+
|cnpj_basico|razao_social                                                         |codigo_natureza_juridica|codigo_qualificacao_responsavel|capital_social|codigo_porte_empresa|ente_federativo_responsavel|
+-----------+---------------------------------------------------------------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+
|08127452   |ELEICAO 2006 WALDEMAR ALBERTO BORGES RODRIGUES NETO DEPUTADO ESTADUAL|4090                    |51                             |0.00          |05                  |NULL                       |
|08127453   |ELEICAO 2006 JOSE ROBERTO DOS SANTOS PALADINI DEPUTADO ESTADUAL      |4090                    |51                             |0.00          |05                  |NULL

In [ ]:
# Obter o tamanho máximo de digitos antes da "," para melhor conveção
df_max_digitos = df_empresas_raw.withColumn(
    "capital_str", f.regexp_replace(f.col("capital_social").cast("string"), "[^0-9]", "")
)

df_max_digitos.selectExpr("max(length(capital_str)) as max_digits").show()

+----------+
|max_digits|
+----------+
|        14|
+----------+



In [98]:
df_empresas_raw = df_empresas_raw.withColumn(
    'capital_social',
    df_empresas_raw['capital_social'] \
        .cast(DecimalType(20, 2)) \
)

df_empresas_raw.orderBy(df_empresas_raw['capital_social'].desc()).show(10, truncate=False)
df_empresas_raw.printSchema()

+-----------+------------------------------------------------+------------------------+-------------------------------+---------------+--------------------+---------------------------+
|cnpj_basico|razao_social                                    |codigo_natureza_juridica|codigo_qualificacao_responsavel|capital_social |codigo_porte_empresa|ente_federativo_responsavel|
+-----------+------------------------------------------------+------------------------+-------------------------------+---------------+--------------------+---------------------------+
|09092574   |SERGIO BOSQUETTI & CIA LTDA ME                  |2062                    |49                             |776158083690.66|01                  |NULL                       |
|07626060   |LGL PINTURAS LTDA                               |2062                    |49                             |532013179015.30|03                  |NULL                       |
|05885737   |TRANSCOL SERVICOS EM TRANSPORTES DE JUNDIAI LTDA|2062         

### Etapa 3: Escrita na Camada Trusted (TRS)

In [99]:
try:
    df_empresas_raw.write.format("delta") \
        .mode("overwrite") \
        .partitionBy("codigo_porte_empresa") \
        .save("../TRS/empresas")
    print("Dados salvos com sucesso na camada Trusted!")
except Exception as e:
    print(f"Erro ao salvar dados: {e}")

25/07/18 12:42:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/07/18 12:42:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/07/18 12:42:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/07/18 12:42:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/18 12:42:51 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/07/18 12:42:51 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/18 12:42:51 WARN MemoryManager: Total allocation exceeds 95.

Dados salvos com sucesso na camada Trusted!


### Etapa 4: Ajuste e Validação

In [122]:
try:
    deltaTable_empresas_trusted = DeltaTable.forPath(spark, "../TRS/empresas")
    df_empresas_trusted = deltaTable_empresas_trusted.toDF()
    df_empresas_trusted.show(5, truncate=False)
    df_empresas_trusted.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+-----------+-----------------------------------------------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+
|cnpj_basico|razao_social                                         |codigo_natureza_juridica|codigo_qualificacao_responsavel|capital_social|codigo_porte_empresa|ente_federativo_responsavel|
+-----------+-----------------------------------------------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+
|06609655   |JOSE LEURIBERTO MAIA                                 |2135                    |50                             |0.00          |05                  |NULL                       |
|06670465   |ELEICOES 2004 - MARIANGELA RIBEIRO LEANDRO - VEREADOR|4090                    |51                             |0.00          |05                  |NULL                       |
|06789422   |ABM - NEGOCIOS IMOBILIARIOS LTDA          

In [121]:
deltaTable_empresas_trusted.update(
    condition = f.col("ente_federativo_responsavel") == "desconhecido",
    set = { "ente_federativo_responsavel": f.lit(None) }
)

25/07/18 23:36:33 WARN UpdateCommand: Could not validate number of records due to missing statistics.


## TRUSTED - Estabelecimentos

###  Etapa 1: Leitura da Camada RAW

In [ ]:
try:
    deltaTable_estabelecimentos_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/estabelecimentos")
    df_estabelecimentos_raw = deltaTable_estabelecimentos_raw.toDF() 
    df_estabelecimentos_raw.show(5, truncate=False)
    df_estabelecimentos_raw.printSchema() 
except Exception as e:
    print(f"Erro na leitura: {e}")

+---------+----------+-------+-------------+---------------------------------+------------------+-----------------------+-------------------------+--------------------+----+---------------------+-----------+-------------------------------------------------------+---------------+----------------------------+------+-----------------------+---------------------------+--------+---+----------------+----+---------+----+---------+-------+----+--------------------------------------+-----------------+----------------------+
|cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|nome_fantasia                    |situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|pais|data_inicio_atividade|cnae_fiscal|cnae_fiscal_secundaria                                 |tipo_logradouro|logradouro                  |numero|complemento            |bairro                     |cep     |uf |codigo_municipio|ddd1|telefone1|ddd2|telefone2|ddd_fax|fax |email                                 |

### Etapa 2: Escrita na Camada TRUSTED (TRS)

In [5]:
try:
    df_estabelecimentos_raw.write.format("delta") \
        .mode("overwrite") \
        .partitionBy("uf") \
        .save("../TRS/estabelecimentos")
    print("Dados salvos com sucesso na camada Trusted!")
except Exception as e:
    print(f"Erro ao salvar dados: {e}")

Dados salvos com sucesso na camada Trusted!


### Etapa 3: Ajuste e Validação

In [74]:
try:
    deltaTable_estabelecimentos_trs = DeltaTable.forPath(spark, "../TRS/estabelecimentos")
    df_estabelecimentos_trs = deltaTable_estabelecimentos_trs.toDF()
    df_estabelecimentos_trs.orderBy(df_estabelecimentos_trs["pais"].desc()).show(5, truncate=False)
    df_estabelecimentos_trs.printSchema()
except Exception as e:
    print(f"Erro: {e}")

+--------------+---------+----------+-------+-------------+-------------+------------------+-----------------------+-------------------------+--------------------+----+---------------------+-----------+----------------------+---------------+----------------------------------+------+-----------+-----------------+----+---+----------------+----+---------+----+---------+-------+----+-----+-----------------+----------------------+
|cnpj_completo |cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|nome_fantasia|situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|pais|data_inicio_atividade|cnae_fiscal|cnae_fiscal_secundaria|tipo_logradouro|logradouro                        |numero|complemento|bairro           |cep |uf |codigo_municipio|ddd1|telefone1|ddd2|telefone2|ddd_fax|fax |email|situacao_especial|data_situacao_especial|
+--------------+---------+----------+-------+-------------+-------------+------------------+-----------------------+------------------------

In [ ]:
try:
    # Convertendo para o tipo date
    df_estabelecimentos_trs = df_estabelecimentos_trs.withColumn(
        "data_situacao_cadastral",
        f.to_date(df_estabelecimentos_trs["data_situacao_cadastral"], 'yyyyMMdd')
    ).withColumn(
        "data_situacao_especial",
        f.to_date(df_estabelecimentos_trs["data_situacao_especial"], 'yyyyMMdd')
    ).withColumn(
        "data_inicio_atividade",
        f.to_date(df_estabelecimentos_trs["data_inicio_atividade"], 'yyyyMMdd')
    )
    df_estabelecimentos_trs.show(5, truncate=False)
    df_estabelecimentos_trs.printSchema()
except Exception as e:
    print(f"Erro na alteração para o tipo data: {e}")

In [ ]:
# Criar coluna cnpj_completo para o estabelecimento
try:
    df_estabelecimentos_trs = df_estabelecimentos_trs.withColumn(
        "cnpj_completo",
        f.concat_ws("", "cnpj_base", "cnpj_ordem", "cnpj_dv")
    )
    df_estabelecimentos_trs.show(5, truncate=False)
    df_estabelecimentos_trs.printSchema()
except Exception as e:
    print(f"Erro na alteração para o tipo data: {e}")

{"ts": "2025-07-19 13:59:22.640", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `codigo_pais` cannot be resolved. Did you mean one of the following? [`ddd_fax`, `pais`, `bairro`, `cep`, `cnpj_base`]. SQLSTATE: 42703", "context": {"file": "line 8 in cell [69]", "line": "", "fragment": "col", "errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o681.withColumn.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `codigo_pais` cannot be resolved. Did you mean one of the following? [`ddd_fax`, `pais`, `bairro`, `cep`, `cnpj_base`]. SQLSTATE: 42703;\n'Project [cnpj_completo#16555, cnpj_base#4637, cnpj_ordem#4638, cnpj_dv#4639, matriz_filial#4640, nome_fantasia#4641, situacao_cadastral#4642, data_situacao_cadastral#4643, motivo_s

Erro na alteração para o tipo data: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `codigo_pais` cannot be resolved. Did you mean one of the following? [`ddd_fax`, `pais`, `bairro`, `cep`, `cnpj_base`]. SQLSTATE: 42703;
'Project [cnpj_completo#16555, cnpj_base#4637, cnpj_ordem#4638, cnpj_dv#4639, matriz_filial#4640, nome_fantasia#4641, situacao_cadastral#4642, data_situacao_cadastral#4643, motivo_situacao_cadastral#4644, nome_cidade_exterior#4645, 'codigo_pais AS pais#16556, data_inicio_atividade#4647, cnae_fiscal#4648, cnae_fiscal_secundaria#4649, tipo_logradouro#4650, logradouro#4651, numero#4652, complemento#4653, bairro#4654, cep#4655, uf#4656, codigo_municipio#4657, ddd1#4658, telefone1#4659, ddd2#4660, ... 6 more fields]
+- Project [concat_ws(, cnpj_base#4637, cnpj_ordem#4638, cnpj_dv#4639) AS cnpj_completo#16555, cnpj_base#4637, cnpj_ordem#4638, cnpj_dv#4639, matriz_filial#4640, nome_fantasia#4641, situacao_cadastral#4642, data_situacao_c

In [11]:
# Reordenar as colunas para deixar o cnpj_completo em primeiro
try:
    colunas_estabelecimentos = df_estabelecimentos_trs.columns
    nova_ordem_estabelecimentos = ['cnpj_completo'] + [col for col in colunas_estabelecimentos if col != 'cnpj_completo']
    df_estabelecimentos_trs = df_estabelecimentos_trs.select(nova_ordem_estabelecimentos)
    df_estabelecimentos_trs.show(5, truncate=False)
    df_estabelecimentos_trs.printSchema()
except Exception as e:
    print(f"Erro na alteração para o tipo data: {e}")

+--------------+---------+----------+-------+-------------+-------------+------------------+-----------------------+-------------------------+--------------------+----+---------------------+-----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+-------------------------+------+-----------+----------------+--------+---+-----------

In [ ]:
try:
    # Atualizando para retirar datas inválidas
    deltaTable_estabelecimentos_trs.update(
        condition=f.length(
            f.col("data_situacao_especial")
        ) < 8,
        set={
            "data_situacao_especial": f.lit(None)
        }
    )
    print("Atualização efetuada!")
except Exception as e:
    print(f"Erro: {e}")

Atualização efetuada!


In [ ]:
try:
    # update por CNPJ
    # deltaTable_estabelecimentos.update(
    #     condition= (f.col("cnpj_base") == "06899799")
    #         & (f.col("cnpj_ordem") == "0001")
    #         & (f.col("cnpj_dv") == "86"),
    #     set={
    #         "nome_fantasia": f.lit("ELEICOES 2004 CF - MANOEL URBANO - AC - COMITE FINANC MUN P/VEREADOR - PMDB".upper()),
    #     }
    # )

    deltaTable_estabelecimentos_trs.update(
        condition=(
            f.col("fax") == None
        ),
        set={
            "ddd_fax": f.lit(None),
        }
    )
    print("Atualização efetuada!")
except Exception as e:
    print(f"Erro: {e}")

Atualização efetuada!


In [ ]:
try:
    # df_estabelecimentos_trusted.filter(
    #     (df_estabelecimentos_trusted["nome_fantasia"] == "IGREJA UNIVERSAL DO REINO DE DEUS")
    #     ).show(truncate=False)

    # Filtrar por CNPJ
    # df_estabelecimentos_trusted.filter(
    #     (f.col("cnpj_base") == "29744778")
    #     & (f.col("cnpj_ordem") == "4750")
    #     & (f.col("cnpj_dv") == "39")
    # ).show(truncate=False)

    # Filtrar por tamanho
    df_estabelecimentos_trs.filter(f.length(df_estabelecimentos_trs["cnae_fiscal"]) != 7).show(truncate=False)

    # Mostrar tudo sem filtro
    df_estabelecimentos_trs.show(truncate=False)
except Exception as e:
    print(f"Erro: {e}")

+---------+----------+-------+-------------+-------------+------------------+-----------------------+-------------------------+--------------------+----+---------------------+-----------+----------------------+---------------+----------+------+-----------+------+---+---+----------------+----+---------+----+---------+-------+---+-----+-----------------+----------------------+
|cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|nome_fantasia|situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|pais|data_inicio_atividade|cnae_fiscal|cnae_fiscal_secundaria|tipo_logradouro|logradouro|numero|complemento|bairro|cep|uf |codigo_municipio|ddd1|telefone1|ddd2|telefone2|ddd_fax|fax|email|situacao_especial|data_situacao_especial|
+---------+----------+-------+-------------+-------------+------------------+-----------------------+-------------------------+--------------------+----+---------------------+-----------+----------------------+---------------+----------+------+

In [75]:
try:
    df_estabelecimentos_trs = df_estabelecimentos_trs.withColumnRenamed("pais", "codigo_pais")
    df_estabelecimentos_trs.show(5, truncate=False)
    df_estabelecimentos_trs.printSchema()
except Exception as e:
    print(f"Erro: {e}")

+--------------+---------+----------+-------+-------------+---------------------------------------+------------------+-----------------------+-------------------------+--------------------+-----------+---------------------+-----------+----------------------+---------------+---------------------------------------+------+------------------+----------------+--------+---+----------------+----+---------+----+---------+-------+----+-----+-----------------+----------------------+
|cnpj_completo |cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|nome_fantasia                          |situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|codigo_pais|data_inicio_atividade|cnae_fiscal|cnae_fiscal_secundaria|tipo_logradouro|logradouro                             |numero|complemento       |bairro          |cep     |uf |codigo_municipio|ddd1|telefone1|ddd2|telefone2|ddd_fax|fax |email|situacao_especial|data_situacao_especial|
+--------------+---------+----------+-------

In [76]:
# Atualizar dados do delta
try:
    df_estabelecimentos_trs.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .partitionBy("uf") \
        .save("../TRS/estabelecimentos")
except Exception as e:
    print(f"Erro: {e}")

## TRUSTED - Sócios

### Etapa 1: Leitura da Camada RAW

In [4]:
try:
    deltaTable_socios_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/socios")
    df_socios_raw: DataFrame = deltaTable_socios_raw.toDF()
    df_socios_raw.show(truncate=False)
    df_socios_raw.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

25/07/18 20:34:04 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+-------------------+------------------------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social             |cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|pais|representante_legal|nome_representante|codigo_qualificacao_representante|faixa_etaria|
+-----------+-------------------+------------------------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
|15508939   |2                  |LUIZ FERNANDO RODRIGUES DE SALES    |***784978**   |49                       |20120423              |NULL|***000000**        |NULL              |00                               |5           |
|15508939   |2                  |LUIZ GONZAGA RODRIGUES DE SALES     |***555608**   |49         

### Etapa 2: Transformação dos dados

In [5]:
try:
    # Verificar se existe divergencia no tamanho das datas
    df_socios_raw.filter(f.length(df_socios_raw["data_entrada_sociedade"]) != 8).show()
except Exception as e:
    print(f"Erro: {e}")

+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social|cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|pais|representante_legal|nome_representante|codigo_qualificacao_representante|faixa_etaria|
+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+



In [10]:
try:
    df_socios_raw = df_socios_raw.withColumn(
        "data_entrada_sociedade",
        f.to_date(df_socios_raw["data_entrada_sociedade"], "yyyyMMdd")
    )
    df_socios_raw.show(truncate=False)
    print(df_socios_raw.select("cnpj_basico").count())
except Exception as e:
    print(f"Erro: {e}")

+-----------+-------------------+------------------------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social             |cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|pais|representante_legal|nome_representante|codigo_qualificacao_representante|faixa_etaria|
+-----------+-------------------+------------------------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
|15508939   |2                  |LUIZ FERNANDO RODRIGUES DE SALES    |***784978**   |49                       |2012-04-23            |NULL|***000000**        |NULL              |00                               |5           |
|15508939   |2                  |LUIZ GONZAGA RODRIGUES DE SALES     |***555608**   |49         

### Etapa 3: Escrita na Camada Trusted

In [7]:
try:
    df_socios_raw.write.format("delta") \
        .mode("overwrite") \
        .save("../TRS/socios")
    print("Escrito com sucesso!")
except Exception as e:
    print(f"Erro na escrita: {e}")

Escrito com sucesso!


### Etapa 4: Ajuste e Validação

In [37]:
# confirmar como esta o delta no TRS
try:
    deltaTable_socios_trusted: DeltaTable = DeltaTable.forPath(spark, "../TRS/socios")
    df_socios_trusted = deltaTable_socios_trusted.toDF()
    df_socios_trusted.show(truncate=False)
    df_socios_trusted.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+-----------+-------------------+---------------------------------+--------------+-------------------------+----------------------+-----------+----------------------------+------------------+---------------------------------+------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social          |cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|codigo_pais|cnpj_cpf_representante_legal|nome_representante|codigo_qualificacao_representante|faixa_etaria|
+-----------+-------------------+---------------------------------+--------------+-------------------------+----------------------+-----------+----------------------------+------------------+---------------------------------+------------+
|03123458   |2                  |JOSE CERQUEIRA COSTA             |***445205**   |49                       |1999-04-30            |NULL       |***000000**                 |NULL              |00                               |6           |
|03123458   |2                  |BONIFACIO D

In [ ]:
# Ajuste no nome das colunas
df_socios_trusted = df_socios_trusted.withColumnRenamed(
    "pais",
    "codigo_pais"
).withColumnRenamed(
    "representante_legal",
    "cnpj_cpf_representante_legal"
)

df_socios_trusted.show()

+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+-----------+----------------------------+------------------+---------------------------------+------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social|cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|codigo_pais|cnpj_cpf_representante_legal|nome_representante|codigo_qualificacao_representante|faixa_etaria|
+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+-----------+----------------------------+------------------+---------------------------------+------------+
|   11441915|                  2|   MARCOS ANTONIO LOPES|   ***563218**|                       49|            2009-12-28|       NULL|                 ***000000**|              NULL|                               00|           6|
|   11442021|                  2|   PAULO ROBERTO ALV...|   ***162865**|            

In [ ]:
# Verificar quantidade de nulos
try:
    df_socios_trusted.select([
        f.sum(f.col(c).isNull().cast("int")).alias(c)
        for c in df_socios_raw.columns
    ]).show()
except Exception as e:
    print(f"Erro: {e}")

+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+-------+-------------------+------------------+---------------------------------+------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social|cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|   pais|representante_legal|nome_representante|codigo_qualificacao_representante|faixa_etaria|
+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+-------+-------------------+------------------+---------------------------------+------------+
|          0|                  0|                    173|          1164|                        0|                     0|2012151|                  0|           1969517|                                0|           0|
+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+-------+-------

In [35]:
# Atualizar dados do delta
try:
    df_socios_trusted.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .save("../TRS/socios")
except Exception as e:
    print(f"Erro: {e}")

## TRUSTED - Simples Nacional

### Etapa 1: Leitura da Camada RAW

In [13]:
try:
    deltaTable_simples_nacional_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/simples_nacional")
    df_simples_nacional_raw = deltaTable_simples_nacional_raw.toDF()
    df_simples_nacional_raw.show(5, truncate=False)
    df_simples_nacional_raw.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|cnpj_basico|opcao_simples|data_opcao_simples|data_exclusao_simples|opcao_mei|data_opcao_mei|data_exclusao_mei|
+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|36536474   |N            |20200303          |20201231             |N        |00000000      |00000000         |
|36536475   |S            |20200303          |00000000             |S        |20200303      |00000000         |
|36536476   |N            |20200303          |20220331             |N        |20200303      |20220331         |
|36536477   |N            |20200303          |20200403             |N        |20200303      |20200403         |
|36536478   |N            |20200303          |20241231             |N        |20200303      |20241231         |
+-----------+-------------+------------------+---------------------+---------+--------------+-----------

### Etapa 2: Transformação dos dados

In [ ]:
# Tratando a data antes de passar para DateType

try:
    df_simples_nacional_raw = df_simples_nacional_raw.withColumn(
        "data_exclusao_simples", f.when(
            df_simples_nacional_raw["data_exclusao_simples"] == "00000000",
            None
        ).otherwise(df_simples_nacional_raw["data_exclusao_simples"])
    ).withColumn(
        "data_opcao_simples", f.when(
            df_simples_nacional_raw["data_opcao_simples"] == "00000000",
            None
        ).otherwise(df_simples_nacional_raw["data_opcao_simples"])
    ).withColumn(
        "data_opcao_mei", f.when(
            df_simples_nacional_raw["data_opcao_mei"] == "00000000",
            None
        ).otherwise(df_simples_nacional_raw["data_opcao_mei"])
    ).withColumn(
        "data_exclusao_mei", f.when(
            df_simples_nacional_raw["data_exclusao_mei"] == "00000000",
            None
        ).otherwise(df_simples_nacional_raw["data_exclusao_mei"])
    )

    print(df_simples_nacional_raw.filter(f.length(df_simples_nacional_raw["data_exclusao_simples"]) != 8).count())
    print(df_simples_nacional_raw.filter(f.length(df_simples_nacional_raw["data_opcao_simples"]) != 8).count())
    print(df_simples_nacional_raw.filter(f.length(df_simples_nacional_raw["data_opcao_mei"]) != 8).count())
    print(df_simples_nacional_raw.filter(f.length(df_simples_nacional_raw["data_exclusao_mei"]) != 8).count())
except Exception as e:
    print(f"Erro: {e}")

0


0


0


0


In [39]:
try:
    df_simples_nacional_raw = df_simples_nacional_raw.withColumn(
        "data_opcao_simples", 
        f.to_date(df_simples_nacional_raw["data_opcao_simples"], "yyyyMMdd")
    ).withColumn(
        "data_exclusao_simples", 
        f.to_date(df_simples_nacional_raw["data_exclusao_simples"], "yyyyMMdd")
    ).withColumn(
        "data_opcao_mei", 
        f.to_date(df_simples_nacional_raw["data_opcao_mei"], "yyyyMMdd")
    ).withColumn(
        "data_exclusao_mei", 
        f.to_date(df_simples_nacional_raw["data_exclusao_mei"], "yyyyMMdd")
    )

    df_simples_nacional_raw.show(5, truncate=False)
except Exception as e:
    print(f"Erro: {e}")

+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|cnpj_basico|opcao_simples|data_opcao_simples|data_exclusao_simples|opcao_mei|data_opcao_mei|data_exclusao_mei|
+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|11968535   |N            |2010-05-17        |2017-12-31           |N        |NULL          |NULL             |
|11968536   |N            |2010-05-20        |2018-02-01           |N        |2010-05-20    |2018-02-01       |
|11968537   |N            |2010-05-20        |2014-02-10           |N        |2010-05-20    |2014-02-10       |
|11968538   |N            |2010-05-20        |2022-02-22           |N        |2010-05-20    |2018-12-31       |
|11968539   |N            |2010-05-20        |2018-01-04           |N        |2010-05-20    |2018-01-04       |
+-----------+-------------+------------------+---------------------+---------+--------------+-----------

### Etapa 3: Escrita na camada Trusted (TRS)

In [40]:
try:
    df_simples_nacional_raw.write.format("delta") \
        .mode("overwrite") \
        .save("../TRS/simples_nacional")
except Exception as e:
    print(f"Erro ao gravar dados: {e}")

### Etapa 4: Ajuste e Validação

In [41]:
# Confirmar alterações, pela camada TRS
try:
    deltaTable_simples_nacional_trusted: DeltaTable = DeltaTable.forPath(spark, "../TRS/simples_nacional")
    df_simples_nacional_trusted = deltaTable_simples_nacional_trusted.toDF()
    df_simples_nacional_trusted.show(5, truncate=False)
    df_simples_nacional_trusted.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|cnpj_basico|opcao_simples|data_opcao_simples|data_exclusao_simples|opcao_mei|data_opcao_mei|data_exclusao_mei|
+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|47096018   |S            |2022-07-11        |NULL                 |S        |2022-07-11    |NULL             |
|47096019   |S            |2022-07-11        |NULL                 |S        |2022-07-11    |NULL             |
|47096020   |N            |2022-07-11        |2025-05-08           |N        |2022-07-11    |2025-05-08       |
|47096021   |N            |2022-07-11        |2024-01-31           |N        |NULL          |NULL             |
|47096022   |N            |2022-07-11        |2023-05-15           |N        |2022-07-11    |2023-05-15       |
+-----------+-------------+------------------+---------------------+---------+--------------+-----------

## TRUSTED - Motivos

### Etapa 1: Leitura da Camada RAW

In [ ]:
try:
    deltaTable_motivos_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/motivos")
    df_motivos_raw: DataFrame = deltaTable_motivos_raw.toDF()
    df_motivos_raw.show(63, truncate=False)
    df_motivos_raw.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+-------------+--------------------------------------------------------------------------------------+
|codigo_motivo|descricao_motivo                                                                      |
+-------------+--------------------------------------------------------------------------------------+
|00           |SEM MOTIVO                                                                            |
|01           |EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA                                       |
|02           |INCORPORACAO                                                                          |
|03           |FUSAO                                                                                 |
|04           |CISAO TOTAL                                                                           |
|05           |ENCERRAMENTO DA FALENCIA                                                              |
|06           |ENCERRAMENTO DA LIQUIDACAO                                

### Etapa 2: Escrita na camada Trusted (TRS)

In [46]:
try:
    df_motivos_raw.write.format("delta") \
        .mode("overwrite") \
        .save("../TRS/motivos")
except Exception as e:
    print(f"Erro ao gravar dados: {e}")

### Etapa 3: Ajuste e Validação

In [38]:
try:
    deltaTable_motivos_trusted: DeltaTable = DeltaTable.forPath(spark, "../TRS/motivos")
    df_motivos_trusted = deltaTable_motivos_trusted.toDF()
    df_motivos_trusted.show(5, truncate=False)
    df_motivos_trusted.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+-------------+-----------------------------------------------+
|codigo_motivo|descricao_motivo                               |
+-------------+-----------------------------------------------+
|00           |SEM MOTIVO                                     |
|01           |EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA|
|02           |INCORPORACAO                                   |
|03           |FUSAO                                          |
|04           |CISAO TOTAL                                    |
+-------------+-----------------------------------------------+
only showing top 5 rows
root
 |-- codigo_motivo: string (nullable = true)
 |-- descricao_motivo: string (nullable = true)



## TRUSTED - Paises

### Etapa 1: Leitura da Camada RAW

In [53]:
try:
    deltaTable_paises_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/paises")
    df_paises_raw = deltaTable_paises_raw.toDF()
    df_paises_raw.show(5, truncate=False)
    print(f"Total de Paises: {df_paises_raw.count()}")
    df_paises_raw.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+------------------------+---------------------+
|codigo_natureza_juridica|natureza_juridica    |
+------------------------+---------------------+
|000                     |COLIS POSTAUX        |
|013                     |AFEGANISTAO          |
|017                     |ALBANIA              |
|020                     |ALBORAN-PEREJIL,ILHAS|
|023                     |ALEMANHA             |
+------------------------+---------------------+
only showing top 5 rows
Total de Paises: 255
root
 |-- codigo_natureza_juridica: string (nullable = true)
 |-- natureza_juridica: string (nullable = true)



### Etapa 2: Transformação dos dados

In [56]:
try:
    df_paises_raw = df_paises_raw.withColumnsRenamed(
        {
            "codigo_natureza_juridica": "codigo_pais",
            "natureza_juridica": "pais"
        },
    )

    df_paises_raw.show(5, truncate=False)
except Exception as e:
    print(f"Erro na leitura: {e}")

+-----------+---------------------+
|codigo_pais|pais                 |
+-----------+---------------------+
|000        |COLIS POSTAUX        |
|013        |AFEGANISTAO          |
|017        |ALBANIA              |
|020        |ALBORAN-PEREJIL,ILHAS|
|023        |ALEMANHA             |
+-----------+---------------------+
only showing top 5 rows


### Etapa 3: Escrita na camada Trusted (TRS)

In [57]:
try:
    df_paises_raw.write.format("delta") \
        .mode("overwrite") \
        .save("../TRS/paises")
    print("Dados salvo com sucesso!")
except Exception as e:
    print(f"Erro na leitura: {e}")

Dados salvo com sucesso!


### Etapa 4: Ajuste e Validação

In [58]:
try:
    deltaTable_paises_trusted: DeltaTable = DeltaTable.forPath(spark, "../TRS/paises")
    df_paises_trusted = deltaTable_paises_trusted.toDF()
    df_paises_trusted.show(5, truncate=False)
    print(f"Total de Paises: {df_paises_raw.count()}")
    df_paises_raw.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+-----------+---------------------+
|codigo_pais|pais                 |
+-----------+---------------------+
|000        |COLIS POSTAUX        |
|013        |AFEGANISTAO          |
|017        |ALBANIA              |
|020        |ALBORAN-PEREJIL,ILHAS|
|023        |ALEMANHA             |
+-----------+---------------------+
only showing top 5 rows
Total de Paises: 255
root
 |-- codigo_pais: string (nullable = true)
 |-- pais: string (nullable = true)



## TRUSTED - Municipios Receita Federal

In [114]:
# Carrega dados da camada RAW
deltaTable_municipios_rf = DeltaTable.forPath(spark, "../RAW/municipios_rf")
df_municipios_rf_raw = deltaTable_municipios_rf.toDF()
df_municipios_rf_raw.show(truncate=False)

+------------+------------------------+
|id_municipio|municipio               |
+------------+------------------------+
|0001        |GUAJARA-MIRIM           |
|0002        |ALTO ALEGRE DOS PARECIS |
|0003        |PORTO VELHO             |
|0004        |BURITIS                 |
|0005        |JI-PARANA               |
|0006        |CHUPINGUAIA             |
|0007        |ARIQUEMES               |
|0008        |CUJUBIM                 |
|0009        |CACOAL                  |
|0010        |NOVA UNIAO              |
|0011        |PIMENTA BUENO           |
|0012        |PARECIS                 |
|0013        |VILHENA                 |
|0014        |PIMENTEIRAS DO OESTE    |
|0015        |JARU                    |
|0016        |PRIMAVERA DE RONDONIA   |
|0017        |OURO PRETO DO OESTE     |
|0018        |SAO FELIPE D'OESTE      |
|0019        |PRESIDENTE MEDICI       |
|0020        |SAO FRANCISCO DO GUAPORE|
+------------+------------------------+
only showing top 20 rows


In [115]:
try:
    df_municipios_rf_raw.write.format("delta") \
        .mode("overwrite") \
        .save("../TRS/municipios_rf")
    print("Dados salvo!")
except Exception as e:
    print(e)

Dados salvo!


In [117]:
try:
    deltaTable_municipios_rf: DeltaTable = DeltaTable.forPath(spark, "../TRS/municipios_rf")
    df_municipios_rf_trusted: DataFrame = deltaTable_municipios_rf.toDF()
    df_municipios_rf_trusted.show(5, truncate=False)
    df_municipios_rf_trusted.printSchema()
except Exception as e:
    print(f"Erro: {e}")

+------------+-----------------------+
|id_municipio|municipio              |
+------------+-----------------------+
|0001        |GUAJARA-MIRIM          |
|0002        |ALTO ALEGRE DOS PARECIS|
|0003        |PORTO VELHO            |
|0004        |BURITIS                |
|0005        |JI-PARANA              |
+------------+-----------------------+
only showing top 5 rows
root
 |-- id_municipio: string (nullable = true)
 |-- municipio: string (nullable = true)



## TRUSTED - Criar Tabela CNPJ

In [ ]:
deltaTable_estabelecimentos_trs.alias("estabelecimentos").merge(
    df_municipios_trusted.alias("municipios"),
    "estabelecimentos.codigo_municipio = municipios.id_municipio"
)

In [136]:
# df_joined = df_1.join(df_2, join_condition, "inner")
# df_estabelecimentos_trusted.show(1, truncate=False)
# df_municipios_rf_trusted.show(1, truncate=False)
df_socios_trusted.show(1, truncate=False)
# df_cnae_trusted.show(1, truncate=False)
# df_municipios_trusted.show(1, truncate=False)
# df_empresas_trusted.show(1, truncate=False)

+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social|cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|pais|representante_legal|nome_representante|codigo_qualificacao_representante|faixa_etaria|
+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
|11441915   |2                  |MARCOS ANTONIO LOPES   |***563218**   |49                       |2009-12-28            |NULL|***000000**        |NULL              |00                               |6           |
+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+----+-------------------+--

In [ ]:
df_empresa = df_empresas_trusted.select(
    "cnpj_basico",
    "razao_social",
    "codigo_natureza_juridica",
    "codigo_qualificacao_responsavel",
    "capital_social",
    "codigo_porte_empresa",
    "ente_federativo_responsavel"
)

df_estabeleciemento: DataFrame = df_estabelecimentos_trs.select(
    f.concat(
        df_estabelecimentos_trs["cnpj_base"],
        df_estabelecimentos_trs["cnpj_ordem"],
        df_estabelecimentos_trs["cnpj_dv"],
    ).alias("cnpj"),
    df_estabelecimentos_trs["nome_fantasia"],
    df_estabelecimentos_trs["situacao_cadastral"],
    df_estabelecimentos_trs["data_situacao_cadastral"],
    df_estabelecimentos_trs["motivo_situacao_cadastral"],
    df_estabelecimentos_trs["nome_cidade_exterior"],
    df_estabelecimentos_trs["data_inicio_atividade"],
    df_estabelecimentos_trs["cnae_fiscal"].alias("cnae_principal"),
    df_estabelecimentos_trs["cnae_fiscal_secundaria"].alias(
        "cnae_secundario"),
    f.concat(
        df_estabelecimentos_trs["tipo_logradouro"],
        f.lit(" "),
        df_estabelecimentos_trs["logradouro"],
        f.lit(", N° "),
        df_estabelecimentos_trs["numero"],
    ).alias("endereco"),
    df_estabelecimentos_trs["cep"],
    df_estabelecimentos_trs["uf"],
    df_estabelecimentos_trs["codigo_municipio"],
    f.concat(
        f.lit("("),
        df_estabelecimentos_trs["ddd1"],
        f.lit(") "),
        df_estabelecimentos_trs["telefone1"],
        f.lit(" - "),
        df_estabelecimentos_trs["email"],
    ).alias("contato")
).filter(df_estabelecimentos_trs["nome_fantasia"].isNotNull())

df_socio: DataFrame = df_socios_trusted.select(
    "cnpj_basico",
    "identificador_socio",
    df_socios_trusted["nome_socio_razao_social"].alias("nome_socio")
)

df_municipio: DataFrame = df_municipios_rf_trusted.select(
    "id_municipio",
    df_municipios_trusted["municipio"]
)


# df_estabeleciemento.show(truncate=False)
# df_municipio.show(truncate=False)
df_estabeleciemento.show(truncate=False)

+--------------+----------------------------------------------------+------------------+-----------------------+-------------------------+--------------------+---------------------+--------------+-----------------------------------------------+--------------------------------------+--------+---+----------------+-----------------------------------------------+
|cnpj          |nome_fantasia                                       |situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|data_inicio_atividade|cnae_principal|cnae_secundario                                |endereco                              |cep     |uf |codigo_municipio|contato                                        |
+--------------+----------------------------------------------------+------------------+-----------------------+-------------------------+--------------------+---------------------+--------------+-----------------------------------------------+--------------------------------

In [ ]:
# Recuperar nome do municipio

df_municipio_condicao = df_municipios_trusted["id_municipio"] == df_estabelecimentos_trs["codigo_municipio"]

df_municipio_final = df_municipios_trusted.join(df_estabelecimentos_trs, df_municipio_condicao, "inner")
df_municipio_final.show(1)

+------------+--------------+---------------+-----------------+--------------+----------------+-----+--------+-------+---------+------------+-----------+---------+----------+-------+-------------+-------------+------------------+-----------------------+-------------------------+--------------------+----+---------------------+-----------+----------------------+---------------+----------+------+-----------+------+---+---+----------------+----+---------+----+---------+-------+---+-----+-----------------+----------------------+
|id_municipio|nome_municipio|id_microrregiao|nome_microrregiao|id_mesorregiao|nome_mesorregiao|id_uf|sigla_uf|nome_uf|id_regiao|sigla_regiao|nome_regiao|cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|nome_fantasia|situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|pais|data_inicio_atividade|cnae_fiscal|cnae_fiscal_secundaria|tipo_logradouro|logradouro|numero|complemento|bairro|cep| uf|codigo_municipio|ddd1|telefone1|ddd2|telefone2|

In [ ]:
# teste join

join_condicao = df_estabelecimentos_trs["cnae_fiscal"] == df_cnae_trusted["codigo_cnae"]
df_joined = df_estabelecimentos_trs.join(df_cnae_trusted, join_condicao, "inner")
df_joined.select("nome_fantasia", "descricao_cnae") \
    .filter(df_joined["nome_fantasia"].isNotNull()) \
    .show(truncate=False)

# df_estabelecimentos_trusted.filter(df_estabelecimentos_trusted["cnae_fiscal"] == "4930201").select("cnae_fiscal").show()
# df_cnae_trusted.filter(df_cnae_trusted["codigo_cnae"] == "4930201").show()

+----------------------------------------------------+-------------------------------------------------------------------------------------------------------------+
|nome_fantasia                                       |descricao_cnae                                                                                               |
+----------------------------------------------------+-------------------------------------------------------------------------------------------------------------+
|ESPACO PATRICIO                                     |Cabeleireiros, manicure e pedicure                                                                           |
|POUSO DA CORUJA                                     |Outros alojamentos não especificados anteriormente                                                           |
|DECORFLEX                                           |Comércio varejista de materiais de construção não especificados anteriormente                                |
|BEE INSUR

## TRUSTED - Joins para consolidado

In [116]:
# Recuperando as delta table

deltaTable_estabelecimentos: DeltaTable = DeltaTable.forPath(spark, "../TRS/estabelecimentos")
deltaTable_empresas: DeltaTable = DeltaTable.forPath(spark, "../TRS/empresas")
deltaTable_municipios: DeltaTable = DeltaTable.forPath(spark, "../TRS/municipios_rf")
deltaTable_municipios_ibge: DeltaTable = DeltaTable.forPath(spark, "../TRS/municipios")
deltaTable_cnae: DeltaTable = DeltaTable.forPath(spark, "../TRS/cnae")
deltaTable_natureza_juridica: DeltaTable = DeltaTable.forPath(spark, "../TRS/natureza_juridica")
deltaTable_simples_nacional: DeltaTable = DeltaTable.forPath(spark, "../TRS/simples_nacional")

In [117]:
# Transformando em DataFrame

df_estabelecimentos: DataFrame = deltaTable_estabelecimentos.toDF()
df_empresas: DataFrame = deltaTable_empresas.toDF()
df_municipios_rf: DataFrame = deltaTable_municipios.toDF()
df_municipios_ibge: DataFrame = deltaTable_municipios_ibge.toDF()
df_cnae: DataFrame = deltaTable_cnae.toDF()
df_natureza: DataFrame = deltaTable_natureza_juridica.toDF()
df_simples: DataFrame = deltaTable_simples_nacional.toDF()

In [124]:
# Renomeando coluna para evitar conflito

df_empresas = df_empresas.withColumnRenamed("codigo_natureza_juridica", "cod_natureza_juridica")

In [125]:
# join da tabela empresas com natureza juridica para ter acesso a descrição de natureza juridica

df_empresas_joined_natureza_juridica: DataFrame = (
    df_empresas
    .join(df_natureza, df_empresas.cod_natureza_juridica == df_natureza.codigo_natureza_juridica, how="left")
)

In [126]:
# Resultado do join anterior

df_empresas_joined_natureza_juridica.show(2, truncate=False)
df_empresas_joined_natureza_juridica.printSchema()

+-----------+-----------------------------------------------------+---------------------+-------------------------------+--------------+--------------------+---------------------------+------------------------+----------------------------------+
|cnpj_basico|razao_social                                         |cod_natureza_juridica|codigo_qualificacao_responsavel|capital_social|codigo_porte_empresa|ente_federativo_responsavel|codigo_natureza_juridica|natureza_juridica                 |
+-----------+-----------------------------------------------------+---------------------+-------------------------------+--------------+--------------------+---------------------------+------------------------+----------------------------------+
|06609655   |JOSE LEURIBERTO MAIA                                 |2135                 |50                             |0.00          |05                  |NULL                       |2135                    |Empresário (Individual)           |
|06670465   |ELE

In [127]:
# Renomeando coluna para evitar conflito

df_empresas_joined_natureza_juridica = df_empresas_joined_natureza_juridica.withColumnRenamed("cnpj_basico", "empresa_cnpj_basico")

In [128]:
# Construção da tabela fato conforme solicitado

df_tabela_fato: DataFrame = (
    df_estabelecimentos
    .join(df_cnae, df_estabelecimentos.cnae_fiscal == df_cnae.codigo_cnae, how="left")
    .join(df_municipios_rf, df_estabelecimentos.codigo_municipio == df_municipios_rf.id_municipio, how="left")
    .join(df_empresas_joined_natureza_juridica, df_estabelecimentos.cnpj_base == df_empresas_joined_natureza_juridica.empresa_cnpj_basico, how='left')
    .join(df_simples, df_estabelecimentos.cnpj_base == df_simples.cnpj_basico, how="left")
)

In [129]:
# Visualização do Schema

df_tabela_fato.printSchema()

root
 |-- cnpj_completo: string (nullable = true)
 |-- cnpj_base: string (nullable = true)
 |-- cnpj_ordem: string (nullable = true)
 |-- cnpj_dv: string (nullable = true)
 |-- matriz_filial: string (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: string (nullable = true)
 |-- data_situacao_cadastral: date (nullable = true)
 |-- motivo_situacao_cadastral: string (nullable = true)
 |-- nome_cidade_exterior: string (nullable = true)
 |-- codigo_pais: string (nullable = true)
 |-- data_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal: string (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- codigo_municipio: string (nullable = true)
 |-- ddd1:

In [130]:
# Resultado visual dos dados

df_tabela_fato.show(2, truncate=False)

+--------------+---------+----------+-------+-------------+-------------+------------------+-----------------------+-------------------------+--------------------+-----------+---------------------+-----------+----------------------+---------------+-------------+------+-----------+----------+--------+---+----------------+----+---------+----+---------+-------+----+------------------------------+-----------------+----------------------+-----------+------------------------------------------------------------------------+------------+---------+-------------------+------------+---------------------+-------------------------------+--------------+--------------------+---------------------------+------------------------+-----------------+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|cnpj_completo |cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|nome_fantasia|situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cid

In [131]:
# Salvar tabela fato para analise posterior

try:
    df_tabela_fato.write.format("delta") \
        .mode("overwrite") \
        .partitionBy("uf") \
        .save("../CONSOLIDADO/01_TABELA_FATO")
except Exception as e:
    print(f"Erro ao gravar: {e}")

In [97]:
# Selecionando partes importantes
df_tabela_fato.select(
    df_tabela_fato["cnpj_completo"].alias("CNPJ"),
    df_tabela_fato["nome_fantasia"].alias("NOME FANTASIA"),
    df_tabela_fato["situacao_cadastral"].alias("SITUAÇÃO CADASTRAL"),
    df_tabela_fato["cnae_fiscal"].alias("CNAE PRINCIPAL"),
    f.concat(
        df_tabela_fato["tipo_logradouro"],
        f.lit(" "),
        df_tabela_fato["logradouro"],
        f.lit(", "),
        df_tabela_fato["numero"],
    ).alias("ENDEREÇO")
).filter(df_tabela_fato["nome_fantasia"].isNotNull()).show(2, truncate=False)

+--------------+------------------+------------------+--------------+------------------------------+
|CNPJ          |NOME FANTASIA     |SITUAÇÃO CADASTRAL|CNAE PRINCIPAL|ENDEREÇO                      |
+--------------+------------------+------------------+--------------+------------------------------+
|18812776000171|MERCEARIA SAO JOSE|08                |8888888       |RUA ALFREDO DE CASTRO, 2      |
|31679901000185|WEDO              |02                |8599604       |RUA DIVA PRADO DE AZEVEDO, 111|
+--------------+------------------+------------------+--------------+------------------------------+
only showing top 2 rows


## Viagem no tempo (Time Travel)

In [ ]:
deltaTable_estabelecimentos_trs.history().show()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|     22|2025-07-18 18:56:...|  NULL|    NULL|    WRITE|{mode -> Overwrit...|NULL|    NULL|     NULL|         21|  Serializable|        false|{numFiles -> 75, ...|        NULL|Apache-Spark/4.0....|
|     21|2025-07-18 18:46:...|  NULL|    NULL|   UPDATE|{predicate -> ["N...|NULL|    NULL|     NULL|         20|  Serializable|        false|{numRemovedFiles ...|        NULL|Apache-Spark/4.0....|
|     20|2

In [ ]:

# Ver versões espeicificas
spark.read.format("delta") \
    .option("versionAsOf", 16) \
    .load("../TRS/estabelecimentos") \
    .show()

+---------+----------+-------+-------------+--------------------+------------------+-----------------------+-------------------------+--------------------+----+---------------------+-----------+----------------------+---------------+--------------------+------+--------------------+--------------------+--------+---+----------------+----+---------+----+---------+-------+--------+--------------------+-----------------+----------------------+
|cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|       nome_fantasia|situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|pais|data_inicio_atividade|cnae_fiscal|cnae_fiscal_secundaria|tipo_logradouro|          logradouro|numero|         complemento|              bairro|     cep| uf|codigo_municipio|ddd1|telefone1|ddd2|telefone2|ddd_fax|     fax|               email|situacao_especial|data_situacao_especial|
+---------+----------+-------+-------------+--------------------+------------------+-----------------------+------

In [ ]:
# Restaurar versão

# deltaTable.restoreToVersion(16)

25/07/18 18:18:47 WARN DAGScheduler: Broadcasting large task binary with size 1119.4 KiB


DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

## Finalizar Spark

In [132]:
spark.stop()